# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [2]:
!ls ../../data

mslr-web10k  vkmarcoranking-v1-document


In [3]:
!ls -lh ../../data/mslr-web10k/

total 20K
drwxrwxr-x 2 jovyan jovyan 4.0K Apr 20 16:04 Fold1
drwxrwxr-x 2 jovyan jovyan 4.0K Apr 20 16:04 Fold2
drwxrwxr-x 2 jovyan jovyan 4.0K Apr 20 16:03 Fold3
drwxrwxr-x 2 jovyan jovyan 4.0K Apr 20 16:03 Fold4
drwxrwxr-x 2 jovyan jovyan 4.0K Apr 20 16:04 Fold5


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [4]:
!ls -lh ../../data/mslr-web10k/Fold1

total 1.3G
-rw-r--r-- 1 jovyan jovyan 267M Apr 20 16:04 test.txt
-rw-r--r-- 1 jovyan jovyan 800M Apr 20 16:04 train.txt
-rw-r--r-- 1 jovyan jovyan 261M Apr 20 16:04 vali.txt


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [5]:
!head -n 1 ../../data/mslr-web10k/Fold1/train.txt

2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [6]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [ ]:
fold_dir = pathlib.Path("../../data/mslr-web10k/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file ../../data/mslr-web10k/Fold1/train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ../../data/mslr-web10k/Fold1/vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ../../data/mslr-web10k/Fold1/test.txt
Dataset loaded from fold_dir ../../data/mslr-web10k/Fold1


Посмотрим на данные:

In [ ]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [ ]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB
None


235 тыс. документов в валидации:

In [ ]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB
None


И 241 тыс. документов в тесте:

In [ ]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB
None


Сколько у нас запросов?

In [12]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [23]:
import itertools
from catboost import CatBoostRanker, Pool


class Model:
    def __init__(self, task_type='CPU'):
        self.task_type = task_type
        self.best_score = 0.0  # Инициализируем с бесконечностью, так как ищем минимум
        self.best_params = {}

    @staticmethod
    def to_catboost_dataset(df):
        y = df['label'].to_numpy()                       # Label: [0-4]
        q = df['qid'].to_numpy().astype('uint32')        # Query Id
        X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
        return (X, y, q)

    def fit(self, df_train, df_valid, params=None):
        X_train, y_train, q_train = self.to_catboost_dataset(df_train)
        train_pool = Pool(data=X_train, label=y_train, group_id=q_train)

        X_valid, y_valid, q_valid = self.to_catboost_dataset(df_valid)
        valid_pool = Pool(data=X_valid, label=y_valid, group_id=q_valid)
        if params is None:
            param_grid = {
                'loss': ['YetiRank', 'QueryRMSE'],
                'learning_rate': [0.03, 0.07],
                'max_bin': [128, 254],
                'depth': [6, 10, 12],
            }
            all_combinations = list(itertools.product(*list(param_grid.values())))
    
            for combination in all_combinations:
                params = {
                    'loss_function': combination[0],
                    'random_seed': 42,
                    'iterations': 8000,
                    'learning_rate': combination[1],
                    'max_bin': combination[2],
                    'depth': combination[3],
                    'task_type': self.task_type,
                    'eval_metric': "NDCG:top=10;type=Exp"
                    
                }
                self.model = CatBoostRanker(**params)
                self.model.fit(train_pool, eval_set=valid_pool, verbose=100, use_best_model=True)
                if np.max(self.model.get_evals_result()["validation"]["NDCG:top=10;type=Exp"]) > self.best_score:
                    self.best_score = np.max(self.model.get_evals_result()["validation"]["NDCG:top=10;type=Exp"])
                    self.best_params = params
                print(f"Current best params: {self.best_params}")
        else:
            self.best_params = params

        self.model = CatBoostRanker(**self.best_params)
        self.model.fit(train_pool, eval_set=valid_pool, verbose=100, use_best_model=True)

    def predict(self, df_test):
        X_test, y_test, q_test = self.to_catboost_dataset(df_test)
        test_pool = Pool(data=X_test, label=y_test, group_id=q_test)
        predictions = self.model.predict(test_pool)
        return predictions


Создадим и применим модель:

In [ ]:
# Create model
model = Model(task_type="GPU")

# Fit
start = timer()
model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2832659	best: 0.2832659 (0)	total: 128ms	remaining: 17m 3s
100:	test: 0.4276595	best: 0.4276595 (100)	total: 1.69s	remaining: 2m 12s
200:	test: 0.4504882	best: 0.4504882 (200)	total: 3.25s	remaining: 2m 6s
300:	test: 0.4599742	best: 0.4599742 (300)	total: 4.8s	remaining: 2m 2s
400:	test: 0.4661336	best: 0.4661336 (400)	total: 6.36s	remaining: 2m
500:	test: 0.4730890	best: 0.4730890 (500)	total: 7.93s	remaining: 1m 58s
600:	test: 0.4785313	best: 0.4785313 (600)	total: 9.49s	remaining: 1m 56s
700:	test: 0.4818239	best: 0.4818239 (700)	total: 11s	remaining: 1m 55s
800:	test: 0.4858823	best: 0.4858823 (800)	total: 12.6s	remaining: 1m 53s
900:	test: 0.4877814	best: 0.4877814 (900)	total: 14.2s	remaining: 1m 51s
1000:	test: 0.4893219	best: 0.4893219 (1000)	total: 15.7s	remaining: 1m 50s
1100:	test: 0.4911008	best: 0.4911008 (1100)	total: 17.3s	remaining: 1m 48s
1200:	test: 0.4930421	best: 0.4930522 (1195)	total: 18.9s	remaining: 1m 46s
1300:	test: 0.4941325	best: 0.4941630 (1275)	

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3454527	best: 0.3454527 (0)	total: 103ms	remaining: 13m 44s
100:	test: 0.4546532	best: 0.4546532 (100)	total: 6.9s	remaining: 8m 59s
200:	test: 0.4690362	best: 0.4690362 (200)	total: 13.8s	remaining: 8m 54s
300:	test: 0.4802397	best: 0.4802397 (300)	total: 20.6s	remaining: 8m 47s
400:	test: 0.4863021	best: 0.4863587 (395)	total: 27.5s	remaining: 8m 41s
500:	test: 0.4925901	best: 0.4925901 (500)	total: 34.4s	remaining: 8m 34s
600:	test: 0.4954280	best: 0.4955442 (595)	total: 41.2s	remaining: 8m 27s
700:	test: 0.4981737	best: 0.4981737 (700)	total: 48.1s	remaining: 8m 20s
800:	test: 0.4992507	best: 0.4992933 (795)	total: 55s	remaining: 8m 14s
900:	test: 0.5009591	best: 0.5010058 (885)	total: 1m 1s	remaining: 8m 7s
1000:	test: 0.5019514	best: 0.5021630 (995)	total: 1m 8s	remaining: 8m
1100:	test: 0.5038158	best: 0.5039250 (1095)	total: 1m 15s	remaining: 7m 53s
1200:	test: 0.5047867	best: 0.5049844 (1180)	total: 1m 22s	remaining: 7m 45s
1300:	test: 0.5064481	best: 0.5064481 (130

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.3813925	best: 0.3813925 (0)	total: 145ms	remaining: 19m 21s
100:	test: 0.5010819	best: 0.5010819 (100)	total: 10.2s	remaining: 13m 14s
200:	test: 0.5074701	best: 0.5074701 (200)	total: 20.2s	remaining: 13m 2s
300:	test: 0.5122345	best: 0.5122345 (300)	total: 30.2s	remaining: 12m 51s
400:	test: 0.5134700	best: 0.5135739 (390)	total: 40.2s	remaining: 12m 41s
500:	test: 0.5145482	best: 0.5150688 (420)	total: 50.1s	remaining: 12m 30s
600:	test: 0.5142797	best: 0.5150688 (420)	total: 1m	remaining: 12m 20s
700:	test: 0.5153263	best: 0.5157414 (690)	total: 1m 10s	remaining: 12m 10s
800:	test: 0.5141184	best: 0.5157414 (690)	total: 1m 20s	remaining: 11m 59s
900:	test: 0.5149766	best: 0.5157414 (690)	total: 1m 30s	remaining: 11m 50s
1000:	test: 0.5141950	best: 0.5157414 (690)	total: 1m 40s	remaining: 11m 40s
1100:	test: 0.5139647	best: 0.5157414 (690)	total: 1m 50s	remaining: 11m 30s
1200:	test: 0.5137311	best: 0.5157414 (690)	total: 2m	remaining: 11m 20s
1300:	test: 0.5132668	best: 

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3908396	best: 0.3908396 (0)	total: 407ms	remaining: 54m 19s
100:	test: 0.5045735	best: 0.5045735 (100)	total: 35.1s	remaining: 45m 45s
200:	test: 0.5106915	best: 0.5106915 (200)	total: 1m 9s	remaining: 45m 9s
300:	test: 0.5152330	best: 0.5153584 (295)	total: 1m 44s	remaining: 44m 33s
400:	test: 0.5148759	best: 0.5153584 (295)	total: 2m 19s	remaining: 43m 58s
500:	test: 0.5151853	best: 0.5156784 (475)	total: 2m 53s	remaining: 43m 23s
600:	test: 0.5153569	best: 0.5158866 (560)	total: 3m 28s	remaining: 42m 48s
700:	test: 0.5151275	best: 0.5158866 (560)	total: 4m 3s	remaining: 42m 13s
800:	test: 0.5153886	best: 0.5158866 (560)	total: 4m 38s	remaining: 41m 38s
900:	test: 0.5159860	best: 0.5164095 (880)	total: 5m 12s	remaining: 41m 4s
1000:	test: 0.5153562	best: 0.5164095 (880)	total: 5m 47s	remaining: 40m 29s
1100:	test: 0.5145624	best: 0.5164095 (880)	total: 6m 22s	remaining: 39m 55s
1200:	test: 0.5141576	best: 0.5164095 (880)	total: 6m 57s	remaining: 39m 20s
1300:	test: 0.51316

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3808221	best: 0.3808221 (0)	total: 185ms	remaining: 24m 36s
100:	test: 0.4875489	best: 0.4875489 (100)	total: 6.63s	remaining: 8m 38s
200:	test: 0.5005493	best: 0.5005493 (200)	total: 13s	remaining: 8m 25s
300:	test: 0.5043690	best: 0.5046690 (295)	total: 19.4s	remaining: 8m 16s
400:	test: 0.5073735	best: 0.5073735 (400)	total: 25.8s	remaining: 8m 9s
500:	test: 0.5097180	best: 0.5097180 (500)	total: 32.2s	remaining: 8m 2s
600:	test: 0.5121403	best: 0.5121403 (600)	total: 38.7s	remaining: 7m 56s
700:	test: 0.5134669	best: 0.5134669 (700)	total: 45.1s	remaining: 7m 49s
800:	test: 0.5139930	best: 0.5139930 (800)	total: 51.5s	remaining: 7m 43s
900:	test: 0.5149078	best: 0.5152532 (890)	total: 57.9s	remaining: 7m 36s
1000:	test: 0.5144914	best: 0.5155797 (915)	total: 1m 4s	remaining: 7m 29s
1100:	test: 0.5151849	best: 0.5155797 (915)	total: 1m 10s	remaining: 7m 23s
1200:	test: 0.5153664	best: 0.5157104 (1190)	total: 1m 17s	remaining: 7m 16s
1300:	test: 0.5154186	best: 0.5159262 (

In [ ]:
for seed in [1337]:
    params = {'loss_function': 'QueryRMSE', 'random_seed': seed, 'iterations': 7500, 'learning_rate': 0.01, 'max_bin': 128, 'depth': 10, 'task_type': 'GPU', 'eval_metric': 'NDCG:top=10;type=Exp'}
    model = Model()
    model.fit(df_train, df_valid, params=params)

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3843786	best: 0.3843786 (0)	total: 81.2ms	remaining: 10m 9s
100:	test: 0.4649174	best: 0.4649174 (100)	total: 6.54s	remaining: 7m 58s
200:	test: 0.4783491	best: 0.4783491 (200)	total: 13s	remaining: 7m 50s
300:	test: 0.4877500	best: 0.4879703 (295)	total: 19.4s	remaining: 7m 44s
400:	test: 0.4947765	best: 0.4947765 (400)	total: 25.8s	remaining: 7m 37s
500:	test: 0.4984680	best: 0.4984680 (500)	total: 32.2s	remaining: 7m 29s
600:	test: 0.5004852	best: 0.5004852 (600)	total: 38.6s	remaining: 7m 22s
700:	test: 0.5034260	best: 0.5036388 (685)	total: 45s	remaining: 7m 16s
800:	test: 0.5043121	best: 0.5044907 (770)	total: 51.4s	remaining: 7m 10s
900:	test: 0.5058795	best: 0.5058795 (900)	total: 57.8s	remaining: 7m 3s
1000:	test: 0.5074456	best: 0.5074456 (1000)	total: 1m 4s	remaining: 6m 57s
1100:	test: 0.5083134	best: 0.5084515 (1085)	total: 1m 10s	remaining: 6m 50s
1200:	test: 0.5092884	best: 0.5094781 (1195)	total: 1m 17s	remaining: 6m 44s
1300:	test: 0.5095813	best: 0.5098989 

Теперь, имея предикты, можно посчитать метрики качества:

In [41]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

y_hat_test = model.predict(df_test)

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.511


Ожидаем, что ваша модель покажет результаты выше бейзлайна!

In [42]:
model.model.save_model("final.cbm")

In [49]:
!unzip model.zip

Archive:  model.zip
  inflating: final.cbm               


In [50]:
model = Model()
model.model = CatBoostRanker().load_model("final.cbm")

In [51]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

y_hat_test = model.predict(df_test)

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.511
